In [ ]:
import os
import glob
from datetime import datetime
import shutil
import numpy as np
from pathlib import Path

In [ ]:
from configs.config_object import ConfigObject
from grid_and_bathy import get_dz_grid, get_grid, build_and_save_mitgcm_grid
from surface_forcings import extract_and_save_surface_forcings, download_weather_reanalysis, download_weather_forecast
from initial_conditions import download_profile_idronaut_datalakes, create_initial_temperature_from_measure_profile, parse_alplakes_1d_from_directory
from run_preprocessing import write_data_config_files, write_secchi, copy_template, remove_all_files_and_folders, write_size_config_files
from utils import modify_arguments, convert_binary_files

## Load config

In [ ]:
config = ConfigObject('config.json')
output_folder = './99-output/'

template_folders=[]
for template_folder_name in config.template_folder:
    template_folders.append(os.path.join('./00-template_mitgcm/', template_folder_name))

In [ ]:
parsed_start_date = datetime.strptime(config.start_date, '%Y%m%d')
parsed_end_date = datetime.strptime(config.end_date, '%Y%m%d')
parsed_ref_date = datetime.strptime(config.reference_date, '%Y%m%d')

sim_duration_in_second = (parsed_end_date - parsed_start_date).total_seconds()
start_time_in_second_from_ref_date = (parsed_start_date - parsed_ref_date).total_seconds()
end_time_in_second_from_ref_date = (parsed_end_date - parsed_ref_date).total_seconds()

## Initialize output folder

In [ ]:
remove_all_files_and_folders(output_folder)
copy_template('./00-template_mitgcm/default', output_folder)
for template_folder in template_folders:
    copy_template(template_folder, output_folder)
config.write_metadata_to_file(os.path.join(output_folder, 'metadata_simulation.txt'))

In [ ]:
for file in Path(config.paths.grid_folder_path).glob("*.npy"):
    shutil.copy(file, os.path.join(output_folder, 'grid'))
for file in Path(config.paths.grid_folder_path).glob("*.json"):
    shutil.copy(file, os.path.join(output_folder, 'grid'))
for file in Path(config.paths.grid_folder_path).glob("*.csv"):
    shutil.copy(file, os.path.join(output_folder, 'grid'))

#In case some files need to be corrected for Linux use.
with open('input.txt', 'r') as f:
    content = f.read().replace('\r\n', '\n')

with open('output.txt', 'w', newline='\n') as f:
    f.write(content)

## Get MITgcm grid

In [ ]:
mitgcm_grid = get_grid(config.paths.grid_folder_path)

## Build binary files

**Get bathymetry**

In [ ]:
#To create bathymetry, see notebook 'create_bathy'
binary_data_folder = os.path.join(output_folder, 'binary_data')
shutil.copy(config.paths.bathy_path, os.path.join(binary_data_folder, 'bathy.bin'))

**Get surface forcings**

In [ ]:
buffer = config.weather_download_buffer
os.makedirs(config.paths.raw_weather_folder, exist_ok=True)
if config.weather_model_type == 'reanalysis':    
    download_weather_reanalysis(config.weather_api_base_url, 
                                config.start_date, config.end_date, 
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_weather_folder)
    print('Finished downloading weather reanalysis')
elif config.weather_model_type == 'forecast':
    download_weather_forecast(config.weather_api_base_url, 
                                config.start_date,  
                                mitgcm_grid.lat_grid.min() - buffer, 
                                mitgcm_grid.lon_grid.min() - buffer, 
                                mitgcm_grid.lat_grid.max() + buffer, 
                                mitgcm_grid.lon_grid.max() + buffer, 
                                config.paths.raw_weather_folder)
    print('Finished downloading weather forecasts')

In [ ]:
time_format = 'UTC'
extract_and_save_surface_forcings(binary_data_folder, 
                                  config.start_date, 
                                  config.end_date, 
                                  config.paths.raw_weather_folder, 
                                  mitgcm_grid,
                                  config.a_lw,
                                  config.weather_model_type)

**Convert to little endian if necessary** (for cscs computation)

In [ ]:
# TO DO : implement check to see whether the datatypes are correct
if config.endian_type == 'little_endian':
    convert_binary_files(os.path.join(output_folder, 'binary_data'), '>f8', '<f8')

### Get initial conditions

In [ ]:
dz_grid = get_dz_grid(os.path.join(config.paths.grid_folder_path, 'dz.csv'))
shaped_salt_initial = np.ones(dz_grid.shape) * 0.03 # constant default values for salt

In [ ]:
pickup_number = ""
shaped_temp_initial = None
if config.initialization_type == 'idronaut':
    file_date, raw_ini_temperature = download_profile_idronaut_datalakes(parsed_start_date)
    shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, raw_ini_temperature)
    print('Initial conditions from Idronaut file from ', file_date)

elif config.initialization_type == 'simstrat':
    simstrat_temperature = parse_alplakes_1d_from_directory(rf'./initial_conditions/simstrat_data/{config.lake_name}')
    raw_ini_temperature = simstrat_temperature.sel(time = parsed_start_date, method='nearest')
    shaped_temp_initial = create_initial_temperature_from_measure_profile(dz_grid, raw_ini_temperature)
    print('Initial conditions from Simstrat ', raw_ini_temperature.time.values)

elif config.initialization_type == 'pickup':
    dt_sim_in_second = config.time_step
    pickup_number = f"pickup.{str(int(start_time_in_second_from_ref_date / dt_sim_in_second)).zfill(10)}"
    shaped_temp_initial = np.zeros(dz_grid.shape) # Dummy values, should not be used anyway

print(f'Pickup number = "{pickup_number}"')

### Adapt config files

In [ ]:
modify_arguments('!reference_date!', config.reference_date, os.path.join(output_folder, 'run_config/data.cal'))

In [ ]:
date_obj = datetime.strptime(config.reference_date, '%Y%m%d')
formatted_date = f"{date_obj.year}-{date_obj.month:02d}-{date_obj.day:02d} 0:0:0"

all_postprocessing_scripts = glob.glob(os.path.join(output_folder, 'postprocessing/*.py'))
for script in all_postprocessing_scripts:
    modify_arguments('!formatted_ref_date!', formatted_date, script)
    modify_arguments('!time_step!', config.time_step, script)

In [ ]:
modify_arguments('!start_date!', config.start_date, os.path.join(output_folder, 'run_config/data.exf'))

In [ ]:
write_secchi(os.path.join(output_folder, 'code/swfrac.F'),
             config.secchi)

In [ ]:
write_data_config_files(os.path.join(output_folder, 'run_config/data'), 
                        shaped_temp_initial, 
                        shaped_salt_initial,  
                        start_time_in_second_from_ref_date, 
                        end_time_in_second_from_ref_date,
                        pickup_number,
                        dz_grid, 
                        config.grid_resolution,
                        config.time_step)

In [ ]:
use_exch2=False
if "exch2" in config.template_folder:
    use_exch2=True
    
# Package to ignore cores that are attributed to land. See notebook grid_and_bathy/check_core_allocation to create files land_cores_Px..etc.txt.
# Doc here: https://wiki.math.uwaterloo.ca/fluidswiki/index.php?title=MITgcm_tips#Setting_the_equation_of_state (look for exch2)
# And there: https://mitgcm.readthedocs.io/en/latest/phys_pkgs/exch2.html (not so helpful... don't we love the official documentation of MITgcm?)
if use_exch2:    
    blank_list_path=os.path.join(config.paths.grid_folder_path, f"land_cores_Px{config.Px}_Py{config.Py}.txt")
    with open(blank_list_path, "r") as file:
        nb_blank_cores, blank_list = file.read().splitlines()
        
    Px = (config.Px * config.Py) - int(nb_blank_cores)
    Py = 1
    
    modify_arguments('!Nx!', config.Nx, os.path.join(output_folder, 'run_config/data.exch2'))
    modify_arguments('!Ny!', config.Ny, os.path.join(output_folder, 'run_config/data.exch2'))
    modify_arguments('!blank_list!', blank_list, os.path.join(output_folder, 'run_config/data.exch2'))
    print('Number of core to use:' + str(Px))
else:
    Px = config.Px
    Py = config.Py

    print('Number of core to use:' + str(Px*Py))

In [ ]:
write_size_config_files(os.path.join(output_folder, 'code/SIZE.h'), 
                        Px, 
                        Py, 
                        config.Nx, 
                        config.Ny, 
                        np.count_nonzero(~np.isnan(dz_grid)),
                        int(config.Nx / config.Px),
                        int(config.Ny / config.Py))

## Next steps

- Copy-paste the folders in "99-output" to the cscs server 
- Change #SBATCH --time=00:05:00 in run/submit-daint-short.sh
- Open CSCS putty console:

export MITGCM_ROOTDIR=/scratch/snx3000/aleroqua/MITgcm-checkpoint67z/

cd /scratch/snx3000/aleroqua/xx/build
$MITGCM_ROOTDIR/tools/genmake2 -mods ../code -mpi -enable=mnc -of ${MITGCM_ROOTDIR}/tools/build_options/linux_ia64_cray_archer
make depend
make -j 8

cd ..
rm -r run
mkdir run
cd run
ln -s ../pickup/* .
ln -s ../run_config/* .
cp ../build/mitgcmuv .

sbatch submit-daint-short.sh

### TO DO

In [ ]:
# add selection of pickup file
# add selection of pickup frequency, diagnostic frequency
# add selection of output type (mnc = true or false)
# add automatic change of computing time? (submit-daint)

SWIRL :
- Automatic adaption of SIMTIMESTEP in 'submit-daint-run_and_swirl_nc.sh'
- Pick config_postprocessing.json depending on the computer_config